#  线性回归
##  线性回归概述
线性回归是机器学习中最基本的问题。机器学习实质上是计算机通过对已有数据得学习得到模型实现对未来数据的预测和处理，主要分为监督学习和无监督学习，监督学习又可以分为回归问题与分类问题。所谓回归问题，是指我们希望得到一个模型，其输出值是连续的，比如训练一个模型实现对房价的预测就是一个回归问题（尽管价格最低到分，但我们近似认为它是连续的）。而对于分类问题，那么我们希望得到的模型输出值就是离散的，比如训练一个检测照片中的人是否吸烟的模型，这就是典型的分类问题（可以定义输出0时表示不吸烟，输出1时表示吸烟）。
这里所介绍的线性回归，就是试图通过学习得到一个线性模型实现预测。所谓线性模型，是指通过对样本属性值的线性组合得到输出。比如房价预测问题，这里的属性就是房子的一些固有性质，比如面积，房间的数量等等，输出就是我们对于房价的预测。从几何意义去理解，如果样本是欧式空间中的一些列点，那么线性回归就是试图找到一条直线去拟合这些点。线性模型可以转化成下面的公式:
$$f(x)=w_0x_0+w_1x_1+...w_nx_n=w^TX$$
其中x为属性值，w为参数，我们希望找到一组参数，使得这个模型的输出尽可能接近于真实情况。因此我们需要先量化一下这个模型的输出值与真实情况之间的差异：
$$J = \dfrac{1}{2m}\sum_{i=1}^{m}(f(x^{(i)})-y^{(i)})^2$$
这个式子我们称之为代价函数，这是机器学习中的重要概念，用于衡量我们的模型与输入输出之间的真实映射的差距。因此要想使得我们的模型达到最优，我们需要将参数进行优化，使得代价函数的值降到最低。目前使用较多的优化算法是梯度下降法：
$$w_j:=w_j-\delta\sum_{i=1}^{m}(f(x^{(i)})-y^{(i)})x_j^{(i)}$$
通过这个公式就可以实现对于模型的训练，通过在程序中不断执行循环，就可以不断优化参数，代价函数的形式保证了这是一个凸函数，可以确保收敛到全局最优处。

##  代码实现

先生成数据

In [3]:
#生成数据
import numpy as np
#生成随机数
np.random.seed(1234)
x = np.random.rand(500,3).reshape(500,3)
#构建映射关系，模拟真实的数据待预测值,映射关系为y = 4.2 + 5.7*x1 + 10.8*x2，可自行设置值进行尝试
y = x.dot(np.array([4.2,5.7,10.8])).reshape(500,1)

### 通过sklearn的线性回归模型实现

In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline

# 调用模型
lr = LinearRegression(fit_intercept=True)
# 训练模型
lr.fit(x,y)
print("估计的参数值为：%s" %(lr.coef_))
# 计算R平方
print('R2:%s' %(lr.score(x,y)))
# 任意设定变量，预测目标值
x_test = np.array([2,4,5]).reshape(1,-1)
y_hat = lr.predict(x_test)
print("预测值为: %s" %(y_hat))

估计的参数值为：[[ 4.2  5.7 10.8]]
R2:1.0
预测值为: [[85.2]]


###   通过最小二乘法的矩阵求解实现

In [5]:
class LR_LS():
    def __init__(self):
        self.w = None      
    def fit(self, X, y):
        # 最小二乘法矩阵求解
        
        self.w = np.dot(np.linalg.inv(np.dot(X.T,X)),np.dot(X.T,y))
        
    def predict(self, X):
        # 用已经拟合的参数值预测新自变量
        
        y_pred = np.dot(X,self.w)
        
        return y_pred

if __name__ == "__main__":
    lr_ls = LR_LS()
    lr_ls.fit(x,y)
    print("估计的参数值：%s" %(lr_ls.w))
    x_test = np.array([2,4,5]).reshape(1,-1)
    print("预测值为: %s" %(lr_ls.predict(x_test)))

估计的参数值：[[ 4.2]
 [ 5.7]
 [10.8]]
预测值为: [[85.2]]


 ###  通过梯度下降法实现

In [6]:
class LR_GD():
   def __init__(self):
       self.w = None     
   def fit(self,X,y,alpha=0.02,loss = 1e-10): # 设定步长为0.002,判断是否收敛的条件为1e-10
       y = y.reshape(-1,1) #重塑y值的维度以便矩阵运算
       
       [m,d] = np.shape(X) #自变量的维度
       self.w = np.zeros((d)).reshape(d,1) #将参数的初始值定为0
       tol = 1e5
       #============================= show me your code =======================
       while tol > loss:
           # here
           self.w -= alpha*np.dot((np.dot(X,self.w)-y).T,X).T/m
           tol = np.dot((y-X.dot(self.w)).T,(y-X.dot(self.w)))/m
           
           
       #============================= show me your code =======================
   def predict(self, X):
       # 用已经拟合的参数值预测新自变量
       y_pred = X.dot(self.w)
       return y_pred  

if __name__ == "__main__":
   lr_gd = LR_GD()
   lr_gd.fit(x,y)
   print("估计的参数值为：%s" %(lr_gd.w))
   x_test = np.array([2,4,5]).reshape(1,-1)
   print("预测值为：%s" %(lr_gd.predict(x_test)))

估计的参数值为：[[ 4.20000569]
 [ 5.70002176]
 [10.79997179]]
预测值为：[[85.19995735]]
